In [1]:
import pandas as pd
import numpy as np
import janitor
import unidecode
import pickle
import os
import operator
import functools
import itertools
from typing import Callable

import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
if 'raw_databases.pkl' not in os.listdir('./data/'):
    from quickstart.loader import XlsxDriveLoader

    Loads = XlsxDriveLoader()  # Drive Folder is hardcoded in module, since this is not prone to change
    databases = Loads.content  # Process takes approximately ~3 minutes to run for the first time.
                               # Then the file will be stored in the data/
else:
    with open('./data/raw_databases.pkl', 'rb') as file:
        databases = pickle.load(file)
    print("Loaded data from .pkl file")

Loaded data from .pkl file


In [3]:
(db_name_contaminantes, db_name_meteoorologicas, 
db_name_anotaciones, _) = databases.keys()


get_col_names: Callable[[pd.core.frame.DataFrame], pd.core.indexes.base.Index] = (
                         lambda df: sorted(list(df.columns)))

def shared_structure(db_name: str) -> list:
    """Returns sheets in dict of DataFrames that conain the same fields"""

    df_col_names = [[key, get_col_names(val)] for key, val in databases[db_name].items()]

    grouped = list()
    for _, g in itertools.groupby(df_col_names, operator.itemgetter(1)):
        group = list(g) 
        if len(group) > 1: # Can't be performed with comprehension since g is a generator/iterator
            grouped.append(group)
    
    if len(grouped) == 1: 
        return [sheet_name for sheet_name, _ in functools.reduce(operator.iconcat, grouped, [])] 
        # Expecting there is only one shared_structure across the DataFrames
    else:
        return ... # Not necessary

In [4]:
raw_contaminantes = pd.concat([databases[db_name_contaminantes][sheet] 
                            for sheet in shared_structure(db_name_contaminantes)])

raw_meteorologicas = pd.concat([databases[db_name_meteoorologicas][sheet] 
                            for sheet in shared_structure(db_name_meteoorologicas)])

In [5]:
# Dictionary for categorical values

identifier = (
    databases[db_name_anotaciones]['LEEME']
    .loc[:, ['Flag', 'Hora']]
    .set_index('Flag')
    .dropna(axis='index')
    .squeeze()
    .apply(lambda string: unidecode.unidecode(string).strip().lower())
    .apply(lambda validity: True if validity == 'valida' else False)
    .append(pd.Series({'x': False})) # Record does not appear on DataFrame
    .to_dict()
)

f_identifier = lambda dict_: identifier.get(dict_, True)

/var/folders/35/qszhsmy97qx2lp6jm0vsbypc0000gn/T/ipykernel_34673/3175683663.py:4: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  databases[db_name_anotaciones]['LEEME']


In [6]:
fields = ['parametro', 'Fecha',
          'SO2', 'SO2 b']  # San Pedro identifiers


def munge(df: pd.core.frame.DataFrame, fields: list):
    return (
        df
        .loc[:, fields]
        .pipe(janitor.rename_columns, new_column_names={'parametro': 'factor',
                                                        'Fecha': 'date'})
        .pipe(janitor.process_text, column_name='factor', string_function='strip')
        .pivot(index='date', columns='factor', values=['SO2', 'SO2 b'])
        .convert_dtypes()
        .apply(lambda field: field.where(field > 0, pd.NA) if pd.api.types.is_numeric_dtype(field)
                                    else field.map(f_identifier))  # Remove negative values from numeric fields
                                                                 # And map str typed variables with a dict
        .pipe(janitor.clean_names, strip_underscores='r')
    )


In [7]:
contaminantes = munge(raw_contaminantes, fields)
meteorologicas = munge(raw_meteorologicas, fields)

In [8]:
# There is a Timestamp missing in df metereologicas
set(contaminantes.index).difference(set(meteorologicas.index))

{Timestamp('2019-05-15 20:00:00')}

In [9]:
df_ = contaminantes.join(other=meteorologicas,
                        how='outer')

In [10]:
df_

so2                                       so2_b        \
factor                 co    no   no2   nox  o3 pm10 pm2_5  so2    co    no   
date                                                                          
2017-01-01 00:00:00   1.1  17.0  20.1  37.0   6  163  <NA>  5.0  True  True   
2017-01-01 01:00:00  1.05  19.5  25.1  44.5   5  117  <NA>  3.9  True  True   
2017-01-01 02:00:00  0.91  16.7  22.5  39.0   5   46  <NA>  3.5  True  True   
2017-01-01 03:00:00  0.75  12.5  16.8  29.2   8   36  <NA>  3.4  True  True   
2017-01-01 04:00:00  0.59   6.5   6.7  13.1  12   25  <NA>  3.2  True  True   
...                   ...   ...   ...   ...  ..  ...   ...  ...   ...   ...   
2021-06-30 19:00:00  0.67   4.4  11.3  15.7  17   36  14.0  3.5  True  True   
2021-06-30 20:00:00  0.66   3.9  10.7  14.6  16   41   7.0  3.6  True  True   
2021-06-30 21:00:00  0.64   3.4  11.7  15.1  15   16   7.0  3.4  True  True   
2021-06-30 22:00:00  0.61   3.4   9.7  13.1  16   31   6.0  3.1  True  True   
2021-06-30 23:00:00   0.6   3.5  12.0  15.5  14   37   6.0  3.3  True  True   

                     ...    so2             so2_b                           \
factor               ...   tout  wdr    wsr   prs rainf    rh     sr  tout   
date                 ...                                                     
2017-01-01 00:00:00  ...  20.68  253    4.0  True  True  True  False  True   
2017-01-01 01:00:00  ...  21.59  244    5.4  True  True  True  False  True   
2017-01-01 02:00:00  ...  20.94  260    4.4  True  True  True  False  True   
2017-01-01 03:00:00  ...  20.99  257    3.7  True  True  True  False  True   
2017-01-01 04:00:00  ...  20.89   84    2.0  True  True  True  False  True   
...                  ...    ...  ...    ...   ...   ...   ...    ...   ...   
2021-06-30 19:00:00  ...  22.78   69   27.1  True  True  True   True  True   
2021-06-30 20:00:00  ...  23.57   74   97.5  True  True  True   True  True   
2021-06-30 21:00:00  ...  22.89   79  117.0  True  True  True  False  True   
2021-06-30 22:00:00  ...  22.72   89  117.7  True  True  True  False  True   
2021-06-30 23:00:00  ...  22.79   72  110.9  True  True  True  False  True   

                                 
factor                wdr   wsr  
date                             
2017-01-01 00:00:00  True  True  
2017-01-01 01:00:00  True  True  
2017-01-01 02:00:00  True  True  
2017-01-01 03:00:00  True  True  
2017-01-01 04:00:00  True  True  
...                   ...   ...  
2021-06-30 19:00:00  True  True  
2021-06-30 20:00:00  True  True  
2021-06-30 21:00:00  True  True  
2021-06-30 22:00:00  True  True  
2021-06-30 23:00:00  True  True  

[39394 rows x 30 columns]

In [11]:
df_.describe(include="all").T

count unique   top   freq        mean        std    min  \
      factor                                                              
so2   co      29973.0    NaN   NaN    NaN    1.679246   1.025919   0.01   
      no      31723.0    NaN   NaN    NaN    9.118173   8.916451    0.5   
      no2     32607.0    NaN   NaN    NaN    11.03015     7.8801    0.1   
      nox     32626.0    NaN   NaN    NaN   19.729188  13.361774    0.5   
      o3      37936.0    NaN   NaN    NaN   25.389788  19.092307    1.0   
      pm10    36946.0    NaN   NaN    NaN   60.021545  35.906347    2.0   
      pm2_5   20458.0    NaN   NaN    NaN   18.928187  13.211943    2.0   
      so2     29466.0    NaN   NaN    NaN    4.205708    2.24159    0.5   
so2_b co        39394      2  True  30073         NaN        NaN    NaN   
      no        39394      2  True  31704         NaN        NaN    NaN   
      no2       39394      2  True  32601         NaN        NaN    NaN   
      nox       39394      2  True  32610         NaN        NaN    NaN   
      o3        39394      2  True  38078         NaN        NaN    NaN   
      pm10      39394      2  True  37080         NaN        NaN    NaN   
      pm2_5     39394      2  True  20617         NaN        NaN    NaN   
      so2       39394      2  True  29557         NaN        NaN    NaN   
so2   prs     38538.0    NaN   NaN    NaN  713.374215    1.65867  702.4   
      rainf    1199.0    NaN   NaN    NaN    0.210325   1.385718   0.01   
      rh      38636.0    NaN   NaN    NaN   56.419402  19.152998    1.0   
      sr      20358.0    NaN   NaN    NaN    0.337981   0.268119  0.001   
      tout    38499.0    NaN   NaN    NaN   22.294383   6.885476   0.04   
      wdr     38498.0    NaN   NaN    NaN  113.355681  64.009974    1.0   
      wsr     29532.0    NaN   NaN    NaN    4.867926   6.685375    0.1   
so2_b prs       39393      2  True  39058         NaN        NaN    NaN   
      rainf     39393      2  True  39137         NaN        NaN    NaN   
      rh        39393      2  True  38791         NaN        NaN    NaN   
      sr        39393      2  True  20827         NaN        NaN    NaN   
      tout      39393      2  True  38822         NaN        NaN    NaN   
      wdr       39393      2  True  38718         NaN        NaN    NaN   
      wsr       39393      2  True  29797         NaN        NaN    NaN   

                25%    50%    75%    max  
      factor                              
so2   co        0.8    1.5   2.39   5.79  
      no        3.2    8.2   11.0  161.0  
      no2       5.7    8.8   14.3   82.3  
      nox      12.3   16.9   23.3  198.2  
      o3       10.0   22.0   35.0  153.0  
      pm10     37.0   52.0   75.0  714.0  
      pm2_5     9.0   16.0   25.0  156.0  
      so2       2.9    3.6    4.8   67.4  
so2_b co        NaN    NaN    NaN    NaN  
      no        NaN    NaN    NaN    NaN  
      no2       NaN    NaN    NaN    NaN  
      nox       NaN    NaN    NaN    NaN  
      o3        NaN    NaN    NaN    NaN  
      pm10      NaN    NaN    NaN    NaN  
      pm2_5     NaN    NaN    NaN    NaN  
      so2       NaN    NaN    NaN    NaN  
so2   prs     712.3  713.2  714.3  738.9  
      rainf    0.01   0.02   0.06  17.99  
      rh       42.0   58.0   72.0   92.0  
      sr      0.089  0.291  0.556  0.971  
      tout     17.7  23.12  27.17  40.44  
      wdr      71.0   87.0  122.0  360.0  
      wsr       1.4    4.2    6.4  176.6  
so2_b prs       NaN    NaN    NaN    NaN  
      rainf     NaN    NaN    NaN    NaN  
      rh        NaN    NaN    NaN    NaN  
      sr        NaN    NaN    NaN    NaN  
      tout      NaN    NaN    NaN    NaN  
      wdr       NaN    NaN    NaN    NaN  
      wsr       NaN    NaN    NaN    NaN

In [12]:
# Replace so2 records to NaN where so2_b values are False.

df = (
    df_
    .apply(lambda row: row.so2.where(row.so2_b.astype('bool')), axis=1) # Corrupts data types
    .replace({pd.NA: np.nan}) # convert_dtypes does not operate when having diferent dtypes for
                              # missing values
    .convert_dtypes()
)

In [13]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
factor,,,,,,,,
co,29930.0,1.680393,1.026013,0.010,0.800,1.500,2.390,5.790
no,31699.0,9.119360,8.919003,0.500,3.200,8.200,11.000,161.000
no2,32588.0,11.030517,7.881935,0.100,5.700,8.800,14.300,82.300
nox,32604.0,19.730637,13.365013,0.500,12.300,16.900,23.300,198.200
o3,37917.0,25.395284,19.093992,1.000,10.000,22.000,35.000,153.000
pm10,36919.0,60.000433,35.892953,2.000,37.000,52.000,75.000,714.000
pm2_5,20455.0,18.930229,13.211835,2.000,9.000,16.000,25.000,156.000
so2,29447.0,4.205593,2.241888,0.500,2.900,3.600,4.800,67.400
prs,38527.0,713.374127,1.658729,702.400,712.300,713.200,714.300,738.900


In [14]:
with open('data/dataframe.pkl', 'wb') as file:
            pickle.dump(df, file)